## Creating Spark Session

In [12]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .enableHiveSupport() \
    .getOrCreate()


+--------+--------------------+-----------+
|database|           tableName|isTemporary|
+--------+--------------------+-----------+
| default|hue__tmp_olist_ge...|      false|
+--------+--------------------+-----------+



## Check tables

In [29]:

spark.sql("show tables").show()

+--------+--------------------+-----------+
|database|           tableName|isTemporary|
+--------+--------------------+-----------+
| default|hue__tmp_olist_ge...|      false|
+--------+--------------------+-----------+



In [81]:
df1=spark.sql("SELECT count (*) FROM ecommerce.hue__tmp_olist_geolocation_dataset")



In [82]:
df1.show()


+--------+
|count(1)|
+--------+
| 1000164|
+--------+



In [23]:
df1.printSchema()

root
 |-- geolocation_zip_code_prefix: long (nullable = true)
 |-- geolocation_lat: double (nullable = true)
 |-- geolocation_lng: double (nullable = true)
 |-- geolocation_city: string (nullable = true)
 |-- geolocation_state: string (nullable = true)



## Reading CSV

### customers

In [129]:
df_customers=spark.read.format("csv").option("header", True).option("inferSchema", True).option("delimiter", ",").load("/user/hive/warehouse/loading_zone/olist_customers/olist_customers_dataset.csv")

In [130]:
df_customers.show()

+--------------------+--------------------+------------------------+--------------------+--------------+
|         customer_id|  customer_unique_id|customer_zip_code_prefix|       customer_city|customer_state|
+--------------------+--------------------+------------------------+--------------------+--------------+
|06b8999e2fba1a1fb...|861eff4711a542e4b...|                   14409|              franca|            SP|
|18955e83d337fd6b2...|290c77bc529b7ac93...|                    9790|sao bernardo do c...|            SP|
|4e7b3e00288586ebd...|060e732b5b29e8181...|                    1151|           sao paulo|            SP|
|b2b6027bc5c5109e5...|259dac757896d24d7...|                    8775|     mogi das cruzes|            SP|
|4f2d8ab171c80ec83...|345ecd01c38d18a90...|                   13056|            campinas|            SP|
|879864dab9bc30475...|4c93744516667ad3b...|                   89254|      jaragua do sul|            SC|
|fd826e7cf63160e53...|addec96d2e059c80c...|            

### geolocation

In [83]:
df_geolocation=spark.read.format("csv").option("header", True).option("inferSchema", True).option("delimiter", ",").load("/user/hive/warehouse/loading_zone/olist_geolocation/olist_geolocation_dataset.csv")

In [84]:
df_geolocation.show()

+---------------------------+-------------------+-------------------+----------------+-----------------+
|geolocation_zip_code_prefix|    geolocation_lat|    geolocation_lng|geolocation_city|geolocation_state|
+---------------------------+-------------------+-------------------+----------------+-----------------+
|                       1037| -23.54562128115268| -46.63929204800168|       sao paulo|               SP|
|                       1046|-23.546081127035535| -46.64482029837157|       sao paulo|               SP|
|                       1046| -23.54612896641469| -46.64295148361138|       sao paulo|               SP|
|                       1041|  -23.5443921648681| -46.63949930627844|       sao paulo|               SP|
|                       1035|-23.541577961711493| -46.64160722329613|       sao paulo|               SP|
|                       1012|-23.547762303364266| -46.63536053788448|       são paulo|               SP|
|                       1047|-23.546273112412678| -46.6

In [85]:
df_geolocation.count()

1000163

In [87]:
df_geolocation.head(3)

[Row(geolocation_zip_code_prefix=1037, geolocation_lat=-23.54562128115268, geolocation_lng=-46.63929204800168, geolocation_city='sao paulo', geolocation_state='SP'),
 Row(geolocation_zip_code_prefix=1046, geolocation_lat=-23.546081127035535, geolocation_lng=-46.64482029837157, geolocation_city='sao paulo', geolocation_state='SP'),
 Row(geolocation_zip_code_prefix=1046, geolocation_lat=-23.54612896641469, geolocation_lng=-46.64295148361138, geolocation_city='sao paulo', geolocation_state='SP')]

In [89]:
from pyspark.sql.functions import when

df_geolocation2 = df_geolocation.withColumn("geolocation_city", when(df_geolocation.geolocation_city == "sao paulo", "São Paulo")\
.when(df_geolocation.geolocation_city == "são paulo", "São Paulo")\
.otherwise(df_geolocation.geolocation_city))



In [94]:
df_geolocation2.select("geolocation_city").distinct().count()

8010

In [95]:
df_geolocation2.select("geolocation_city").distinct().show()

+--------------------+
|    geolocation_city|
+--------------------+
|   redencao da serra|
|  aguas de sao pedro|
|            itirapuã|
|           indiaporã|
|         paulistânia|
|        capão bonito|
|              taguaí|
|                iepe|
|         teresópolis|
|              bacaxa|
|divino de sao lou...|
|divino das laranj...|
|            vermelho|
|            caçarema|
|          cacaratiba|
|                pote|
|            camacari|
|           itaberaba|
|              gloria|
|           arapiraca|
+--------------------+
only showing top 20 rows



In [96]:
df_geolocation2.show()

+---------------------------+-------------------+-------------------+----------------+-----------------+
|geolocation_zip_code_prefix|    geolocation_lat|    geolocation_lng|geolocation_city|geolocation_state|
+---------------------------+-------------------+-------------------+----------------+-----------------+
|                       1037| -23.54562128115268| -46.63929204800168|       São Paulo|               SP|
|                       1046|-23.546081127035535| -46.64482029837157|       São Paulo|               SP|
|                       1046| -23.54612896641469| -46.64295148361138|       São Paulo|               SP|
|                       1041|  -23.5443921648681| -46.63949930627844|       São Paulo|               SP|
|                       1035|-23.541577961711493| -46.64160722329613|       São Paulo|               SP|
|                       1012|-23.547762303364266| -46.63536053788448|       São Paulo|               SP|
|                       1047|-23.546273112412678| -46.6

In [99]:
from pyspark.sql.functions import initcap, col
df_geolocation2.select("*", initcap(col('geolocation_city'))).head(50)

[Row(geolocation_zip_code_prefix=1037, geolocation_lat=-23.54562128115268, geolocation_lng=-46.63929204800168, geolocation_city='São Paulo', geolocation_state='SP', initcap(geolocation_city)='São Paulo'),
 Row(geolocation_zip_code_prefix=1046, geolocation_lat=-23.546081127035535, geolocation_lng=-46.64482029837157, geolocation_city='São Paulo', geolocation_state='SP', initcap(geolocation_city)='São Paulo'),
 Row(geolocation_zip_code_prefix=1046, geolocation_lat=-23.54612896641469, geolocation_lng=-46.64295148361138, geolocation_city='São Paulo', geolocation_state='SP', initcap(geolocation_city)='São Paulo'),
 Row(geolocation_zip_code_prefix=1041, geolocation_lat=-23.5443921648681, geolocation_lng=-46.63949930627844, geolocation_city='São Paulo', geolocation_state='SP', initcap(geolocation_city)='São Paulo'),
 Row(geolocation_zip_code_prefix=1035, geolocation_lat=-23.541577961711493, geolocation_lng=-46.64160722329613, geolocation_city='São Paulo', geolocation_state='SP', initcap(geoloc

In [136]:
df_geolocation2.show()

+---------------------------+-------------------+-------------------+----------------+-----------------+
|geolocation_zip_code_prefix|    geolocation_lat|    geolocation_lng|geolocation_city|geolocation_state|
+---------------------------+-------------------+-------------------+----------------+-----------------+
|                       1037| -23.54562128115268| -46.63929204800168|       São Paulo|               SP|
|                       1046|-23.546081127035535| -46.64482029837157|       São Paulo|               SP|
|                       1046| -23.54612896641469| -46.64295148361138|       São Paulo|               SP|
|                       1041|  -23.5443921648681| -46.63949930627844|       São Paulo|               SP|
|                       1035|-23.541577961711493| -46.64160722329613|       São Paulo|               SP|
|                       1012|-23.547762303364266| -46.63536053788448|       São Paulo|               SP|
|                       1047|-23.546273112412678| -46.6

In [103]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

geolocator = Nominatim(user_agent="fiap_mba")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds = 1,   return_value_on_exception = None) 
# adding 1 second padding between calls

In [ ]:
location = geocode.reverse((lat, long))

## Transforming to parquet

In [66]:
customers_parquet= df_customers.write.mode("overwrite").format("parquet").save("/user/hive/warehouse/raw_zone/")

In [68]:
customers_parquet

## Using geopy

In [76]:
from geopy.geocoders import Nominatim

In [77]:
geolocator = Nominatim(user_agent="fiap_mba")

In [78]:
location = geolocator.reverse("-23.545884279214015, -46.643163191240035")

In [79]:
print(location)

Edifício Comandante Linneu Gomes, 86, Avenida São Luís, República, São Paulo, Região Imediata de São Paulo, Região Metropolitana de São Paulo, Região Geográfica Intermediária de São Paulo, São Paulo, Região Sudeste, 01046-000, Brasil


## Using pycep_correios

In [240]:
from pycep_correios import get_address_from_cep, WebService

address = get_address_from_cep('01037000', webservice=WebService.APICEP)

In [241]:
address

{'bairro': 'República',
 'cep': '01037-000',
 'cidade': 'São Paulo',
 'logradouro': 'Rua Conselheiro Crispiniano - lado par',
 'uf': 'SP',
 'complemento': ''}

In [181]:
import pandas as pd

In [182]:
df = pd.DataFrame([
[-87.548627, 41.728184],
[-87.737227, 41.749111],
[-87.743974, 41.924143],
[-87.659294, 41.869314],
 [-87.727808, 41.877007],
], columns=["Longitude", "Latitude"])

In [183]:
df

,Longitude,Latitude
0,-87.548627,41.728184
1,-87.737227,41.749111
2,-87.743974,41.924143
3,-87.659294,41.869314
4,-87.727808,41.877007


In [184]:
from tqdm import tqdm
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

tqdm.pandas()
geolocator = Nominatim(user_agent="fiap_mba2")
reverse = RateLimiter(geolocator.reverse, min_delay_seconds=1)


In [185]:
df

,Longitude,Latitude
0,-87.548627,41.728184
1,-87.737227,41.749111
2,-87.743974,41.924143
3,-87.659294,41.869314
4,-87.727808,41.877007


In [187]:
df['Location'] = df.progress_apply(
lambda row: reverse((row['Latitude'], row['Longitude'])),
axis=1
)

100%|██████████| 5/5 [00:05<00:00,  1.01s/it]


In [188]:
df

,Longitude,Latitude,Location
0,-87.548627,41.728184,"(92nd Street & Baltimore, East 92nd Street, So..."
1,-87.737227,41.749111,"(79th Street & Kilpatrick EB, West 79th Street..."
2,-87.743974,41.924143,"(4701, West Fullerton Avenue, Belmont Cragin, ..."
3,-87.659294,41.869314,"(1301-1307, West Taylor Street, Illinois Medic..."
4,-87.727808,41.877007,"(Jackson & Karlov, West Jackson Boulevard, Wes..."


In [189]:
def parse_zipcode(location):
    if location and location.raw.get('address') and location.raw['address'].get('postcode'):
        return location.raw['address']['postcode']
    else:
        return None

df['Zipcode'] = df['Location'].apply(parse_zipcode)

In [194]:
df_geolocation3=df_geolocation2.toPandas()

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:38495)
Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:38495)

In [195]:
print(df_geolocation3)

         geolocation_zip_code_prefix  geolocation_lat  geolocation_lng  \
0                               1037       -23.545621       -46.639292   
1                               1046       -23.546081       -46.644820   
2                               1046       -23.546129       -46.642951   
3                               1041       -23.544392       -46.639499   
4                               1035       -23.541578       -46.641607   
...                              ...              ...              ...   
1000158                        99950       -28.068639       -52.010705   
1000159                        99900       -27.877125       -52.224882   
1000160                        99950       -28.071855       -52.014716   
1000161                        99980       -28.388932       -51.846871   
1000162                        99950       -28.070104       -52.018658   

        geolocation_city geolocation_state  
0              São paulo                SP  
1              São pa

In [196]:
df_geolocation3['geolocation_city'] = df_geolocation3['geolocation_city'].str.capitalize()

In [142]:
print(df_geolocation3)

         geolocation_zip_code_prefix  geolocation_lat  geolocation_lng  \
0                               1037       -23.545621       -46.639292   
1                               1046       -23.546081       -46.644820   
2                               1046       -23.546129       -46.642951   
3                               1041       -23.544392       -46.639499   
4                               1035       -23.541578       -46.641607   
...                              ...              ...              ...   
1000158                        99950       -28.068639       -52.010705   
1000159                        99900       -27.877125       -52.224882   
1000160                        99950       -28.071855       -52.014716   
1000161                        99980       -28.388932       -51.846871   
1000162                        99950       -28.070104       -52.018658   

        geolocation_city geolocation_state  
0              São paulo                SP  
1              São pa

In [204]:
df_geolocation3 = df_geolocation3.head(20)

In [205]:
df_geolocation3

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,São paulo,SP
1,1046,-23.546081,-46.644820,São paulo,SP
2,1046,-23.546129,-46.642951,São paulo,SP
3,1041,-23.544392,-46.639499,São paulo,SP
4,1035,-23.541578,-46.641607,São paulo,SP
5,1012,-23.547762,-46.635361,São paulo,SP
6,1047,-23.546273,-46.641225,São paulo,SP
7,1013,-23.546923,-46.634264,São paulo,SP
8,1029,-23.543769,-46.634278,São paulo,SP
9,1011,-23.547640,-46.636032,São paulo,SP


In [197]:
from tqdm import tqdm
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter


tqdm.pandas()
geolocator = Nominatim(user_agent="fiap_mba")
reverse = RateLimiter(geolocator.reverse, min_delay_seconds=1)

In [206]:
df_geolocation3['address'] = df_geolocation3.progress_apply(
lambda row: reverse((row['geolocation_lat'], row['geolocation_lng'])),
axis=1
)

100%|██████████| 20/20 [00:19<00:00,  1.04it/s]
/opt/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [208]:
df_geolocation3

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state,address
0,1037,-23.545621,-46.639292,São paulo,SP,"(Casas Bahia, 131, Praça Ramos de Azevedo, Rep..."
1,1046,-23.546081,-46.644820,São paulo,SP,"(303, Avenida Ipiranga, Vila Buarque, Repúblic..."
2,1046,-23.546129,-46.642951,São paulo,SP,"(Confidence Cambio, Calçada Compartilhada Av S..."
3,1041,-23.544392,-46.639499,São paulo,SP,"(Galeria R Monteiro, 77, Rua 24 de Maio, Repúb..."
4,1035,-23.541578,-46.641607,São paulo,SP,"(816, Avenida São João, República, São Paulo, ..."
5,1012,-23.547762,-46.635361,São paulo,SP,"(Edifício George Loeb, 113, Rua da Quitanda, G..."
6,1047,-23.546273,-46.641225,São paulo,SP,"(Edifício Vicentina, Rua Doutor Bráulio Gomes,..."
7,1013,-23.546923,-46.634264,São paulo,SP,"(Edifício Canadá, Rua 15 de Novembro, Glicério..."
8,1029,-23.543769,-46.634278,São paulo,SP,"(Mosteiro de São Bento, Largo São Bento, Glicé..."
9,1011,-23.547640,-46.636032,São paulo,SP,"(30, Praça do Patriarca, Glicério, Sé, São Pau..."


In [210]:
def parse_zipcode(location):
    if location and location.raw.get('address') and location.raw['address'].get('postcode'):
        return location.raw['address']['postcode']
    else:
        return None

df_geolocation3['Zipcode'] = df_geolocation3['address'].progress_apply(parse_zipcode)

100%|██████████| 20/20 [00:00<00:00, 12642.97it/s]
/opt/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [259]:
df_geolocation3

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state,address,Zipcode
0,1037,-23.545621,-46.639292,São paulo,SP,"(Casas Bahia, 131, Praça Ramos de Azevedo, Rep...",01037-000
1,1046,-23.546081,-46.644820,São paulo,SP,"(303, Avenida Ipiranga, Vila Buarque, Repúblic...",01046-010
2,1046,-23.546129,-46.642951,São paulo,SP,"(Confidence Cambio, Calçada Compartilhada Av S...",01046-001
3,1041,-23.544392,-46.639499,São paulo,SP,"(Galeria R Monteiro, 77, Rua 24 de Maio, Repúb...",01041-001
4,1035,-23.541578,-46.641607,São paulo,SP,"(816, Avenida São João, República, São Paulo, ...",01036-100
5,1012,-23.547762,-46.635361,São paulo,SP,"(Edifício George Loeb, 113, Rua da Quitanda, G...",01012-010
6,1047,-23.546273,-46.641225,São paulo,SP,"(Edifício Vicentina, Rua Doutor Bráulio Gomes,...",01043-000
7,1013,-23.546923,-46.634264,São paulo,SP,"(Edifício Canadá, Rua 15 de Novembro, Glicério...",01013-001
8,1029,-23.543769,-46.634278,São paulo,SP,"(Mosteiro de São Bento, Largo São Bento, Glicé...",01029-010
9,1011,-23.547640,-46.636032,São paulo,SP,"(30, Praça do Patriarca, Glicério, Sé, São Pau...",01002-010


In [234]:
df_geolocation3.dtypes

geolocation_zip_code_prefix      int32
geolocation_lat                float64
geolocation_lng                float64
geolocation_city                object
geolocation_state               object
address                         object
Zipcode                         object
dtype: object

In [255]:
s=df_geolocation3['Zipcode'].replace("-","")

AttributeError: module 'pandas' has no attribute 'replace'

In [250]:
s

0     01037-000
1     01046-010
2     01046-001
3     01041-001
4     01036-100
5     01012-010
6     01043-000
7     01013-001
8     01029-010
9     01002-010
10    01013-001
11    01032-001
12    01014-020
13    01003-000
14    01037-912
15    01046-010
16    01039-000
17    01024-100
18    01009-000
19    01046-000
Name: Zipcode, dtype: object

In [251]:
s = pd.Series(df_geolocation3['Zipcode'] )


In [256]:
s=pd.to_numeric(df_geolocation3['Zipcode'],errors='coerce')

ValueError: invalid literal for int() with base 10: '01037-000'

In [239]:
pd.to_numeric(s)

ValueError: Unable to parse string "01037-000" at position 0

In [227]:
from pycep_correios import get_address_from_cep, WebService

df_geolocation3['new_address'] = df_geolocation3.progress_apply(
    lambda row: get_address_from_cep(row['Zipcode'], webservice=WebService.APICEP), axis=1
)

#df.apply(lambda row: row['A'] + row['B'], axis=1)


 55%|█████▌    | 11/20 [00:03<00:03,  2.84it/s]


InvalidCEP: occurred at index 0